In [70]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import requests 
import numpy as np


In [71]:
census = pd.read_csv('./DEC_00_SF1_H007.csv')
census.rename(columns={'Id2':'zip_code','Total:':'total',
                       'Not Hispanic or Latino householder:':'non_hispanic_tot',
                      'Hispanic or Latino householder:':'hispanic_tot'},inplace = True)
income = pd.read_csv('Income_Data.csv')
income.rename(columns={'Estimate; Per capita income in the past 12 months (in 2017 inflation-adjusted dollars)': 
                       'per_capita_income_2017_estimate',
                      'Margin of Error; Per capita income in the past 12 months (in 2017 inflation-adjusted dollars)':
                      'per_capita_income_2017_margin'},inplace = True)

In [72]:
#need to use inner join as there are some 991HH, 991XX zip codes in the census data with no income analysis
zip_income = pd.merge(census,income, on = 'Id',how = 'inner',suffixes =['_income','_zip'])
#delete the puerto rican zip codes
# add back the trailing 0's for zip codes less than 5 digits
zip_income['zip_code'] = zip_income['zip_code'].astype('str')
fun = lambda x : '0'+ str(x) if len(x) == 4 else ('00'+ str(x) if len(x) == 3 else str(x))
zip_income['zip_code'] = zip_income['zip_code'].apply(fun)

In [73]:
#Calculate the total people for each race, regardless of hispanic or not
list_race = ['white_tot','black_tot','indian_tot','asian_tot','native_tot','other_tot','mixed_tot']
i = 1
for race in list_race:
    zip_income[race] = zip_income.iloc[:,3+i] + zip_income.iloc[:,11+i]
    i+= 1
    
zip_income.head()

,Id,zip_code,Geography_income,total,non_hispanic_tot,Not Hispanic or Latino householder: - Householder who is White alone,Not Hispanic or Latino householder: - Householder who is Black or African American alone,Not Hispanic or Latino householder: - Householder who is American Indian and Alaska Native alone,Not Hispanic or Latino householder: - Householder who is Asian alone,Not Hispanic or Latino householder: - Householder who is Native Hawaiian and Other Pacific Islander alone,...,Geography_zip,per_capita_income_2017_estimate,per_capita_income_2017_margin,white_tot,black_tot,indian_tot,asian_tot,native_tot,other_tot,mixed_tot
0,8600000US00601,00601,ZCTA5 00601,5895,27,23,2,0,1,0,...,ZCTA5 00601,7041.0,775.0,5895,5618,67,8,1,0,104
1,8600000US00602,00602,ZCTA5 00602,13520,98,79,6,0,3,0,...,ZCTA5 00602,8978.0,597.0,13520,11908,558,22,8,1,631
2,8600000US00603,00603,ZCTA5 00603,19152,229,185,19,1,16,0,...,ZCTA5 00603,10897.0,819.0,19152,15789,1028,37,33,4,1718
3,8600000US00606,00606,ZCTA5 00606,2013,13,11,1,0,1,0,...,ZCTA5 00606,5960.0,708.0,2013,1812,77,4,1,0,50
4,8600000US00610,00610,ZCTA5 00610,9262,73,61,6,0,5,0,...,ZCTA5 00610,9266.0,542.0,9262,7824,447,23,10,0,524


In [74]:
# subset the data and make sure that the zip code contains 5000 or more people
census_sub = zip_income.loc[(zip_income['total'] >= 5000)&(~zip_income['zip_code'].str.startswith('00')),:]

In [75]:
# randomly select 100 zip code out of subset census data
census_sub_sample = census_sub.sample(n = 100,random_state=100).reset_index(drop = True)


In [76]:
census_sub_sample_short = census_sub_sample.drop(columns=['Geography_income','Id2'])
census_sub_sample_short.head()

,Id,zip_code,total,non_hispanic_tot,Not Hispanic or Latino householder: - Householder who is White alone,Not Hispanic or Latino householder: - Householder who is Black or African American alone,Not Hispanic or Latino householder: - Householder who is American Indian and Alaska Native alone,Not Hispanic or Latino householder: - Householder who is Asian alone,Not Hispanic or Latino householder: - Householder who is Native Hawaiian and Other Pacific Islander alone,Not Hispanic or Latino householder: - Householder who is Some other race alone,...,Geography_zip,per_capita_income_2017_estimate,per_capita_income_2017_margin,white_tot,black_tot,indian_tot,asian_tot,native_tot,other_tot,mixed_tot
0,8600000US01850,01850,5680,5056,4484,204,5,248,0,10,...,ZCTA5 01850,19486.0,1964.0,5680,4737,221,8,249,0,312
1,8600000US99502,99502,6576,6358,5192,226,301,403,24,6,...,ZCTA5 99502,41256.0,2823.0,6576,5292,231,313,407,26,81
2,8600000US33126,33126,15244,946,699,94,8,101,4,5,...,ZCTA5 33126,17570.0,970.0,15244,13432,322,23,105,6,810
3,8600000US10035,10035,11169,5865,483,5033,23,88,1,26,...,ZCTA5 10035,21618.0,1727.0,11169,2426,5594,83,93,8,2402
4,8600000US10029,10029,27465,12873,2688,8557,63,945,2,98,...,ZCTA5 10029,27966.0,1641.0,27465,8159,9960,225,977,18,6592


In [77]:
census_sub_sample_short.to_csv('sample_census2_100.csv')